In [11]:
# Cell 2: 导入库与路径配置
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 设置 Hugging Face 全局缓存目录
os.environ["HF_HOME"] = "/ssd_2t_1/wyq_workspace/hf_cache"
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import pandas as pd

# --- 配置 ---
# Mistral 模型路径 (如果你已经下载到本地，改成本地路径；否则从 HF 拉取)
# 论文使用的是 v0.2 Instruct 版本
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.2" 

# 你的 UMI 映射文件 (Stage 1 产出)
# 格式应该是 CSV: ID, Description, ...
UMI_CSV_PATH = "/home/wyq/GenKubeSec_Reproduce/unify_error_umi/policies_with_remediation.csv"

# 你的检测模型输出结果 (Stage 2 产出，用于测试)
# 假设是一个 jsonl 文件，包含 filename, content, predicted_labels
DETECTION_RESULT_PATH = "/home/wyq/GenKubeSec_Reproduce/test_results_sample.jsonl"

In [12]:
# Cell 3: 加载 UMI 映射表
def load_umi_mapping(csv_path):
    """
    将 CSV 加载为字典: { "52": "Image tag ':latest' used", ... }
    """
    try:
        df = pd.read_csv(UMI_CSV_PATH, dtype=str).fillna("")
        # 假设 CSV 有 'ID' 和 'Checkov_Policy' (或者其他描述列)
        # 这里我们优先选一个描述最清楚的列，或者合并几列
        # 论文中提到使用 unified description
        mapping = {}
        for _, row in df.iterrows():
            uid = row['ID']
            # 这里简单取 Checkov Policy 作为描述，你可以根据实际 CSV 结构调整
            # 如果你有专门生成的 'Description' 列更好
            desc = (
                str(row.get('Checkov_Policy', '')).strip() or 
                str(row.get('Kube_Linter_Policy', '')).strip() or 
                str(row.get('Terrascan_Policy', '')).strip() or 
                str(row.get('Remediation', '')).strip()
            )
            mapping[uid] = desc
        print(f"✅ 成功加载 UMI 映射，共 {len(mapping)} 条规则。")
        return mapping
    except Exception as e:
        print(f"❌ 加载 UMI 失败: {e}")
        return {}

umi_map = load_umi_mapping(UMI_CSV_PATH)

# 测试一下
print(umi_map.get("113"))

✅ 成功加载 UMI 映射，共 165 条规则。
Indicates when a subject (Group/User/ServiceAccount) has create access to Pods. CIS Benchmark 5.1.4: The ability to create pods in a cluster opens up possibilities for privilege escalation and should be restricted, where possible.


In [13]:
os.environ["http_proxy"] = "http://127.0.0.1:36217"
os.environ["https_proxy"] = "http://127.0.0.1:36217"
# Cell 4: 加载 Mistral-7B 模型
print("🚀 正在加载 Mistral-7B-Instruct ...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# 使用 4-bit 量化加载 (速度快，显存占用极低，约 6GB)
# 如果你想追求极致精度，可以去掉 load_in_4bit，改用 torch_dtype=torch.float16
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    load_in_4bit=True,  # 需要安装 bitsandbytes
    torch_dtype=torch.float16
)

print("✅ 模型加载完成！")

🚀 正在加载 Mistral-7B-Instruct ...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ 模型加载完成！


In [26]:
# Cell 5: 改进后的 Prompt 生成函数 (基于论文附录)

def construct_prompt(yaml_content, error_id, error_description):
    """
    根据 GenKubeSec 论文复现 Prompt 模板。
    
    参数:
    - yaml_content: 待检测的 YAML 文件内容
    - error_id: 错误 ID (例如 "LLM_error_150" 或 "150")
    - error_description: 错误描述 (例如 "Indicates when a deployment uses less than three replicas")
    """
    
    # 1. 确保 error_id 格式为 LLM_error_<x>
    # 如果传入的是纯数字 "150"，自动转为 "LLM_error_150"
    if not str(error_id).startswith("LLM_error_"):
        formatted_error_id = f"LLM_error_{error_id}"
    else:
        formatted_error_id = str(error_id)

    # 2. 系统提示词 (System Prompt) - 完全复刻论文内容
    system_instruction = f"""description: As a top DevOps engineer, you are provided with a K8s manifest that contains a misconfiguration labeled as {formatted_error_id}. Your tasks are:
tasks:
- Identify the line of the misconfiguration.
- Provide the text of this line.
- Offer reasoning for the identified misconfiguration only.
- Suggest how to fix each identified misconfiguration.
- Write the misconfiguration number corresponding to {formatted_error_id}.
- Do not suggest a corrected version of the entire manifest.

output format: 
The output must be a single, valid JSON object.
Do not use Markdown code blocks.
Ensure all keys and string values are enclosed in double quotes.

IMPORTANT SYNTAX RULE:
When writing YAML or JSON snippets inside the 'Remediation' field, YOU MUST USE SINGLE QUOTES (') for keys and values. NEVER use double quotes (") inside the value string, as it breaks the JSON structure.
Example: Use "add: ['NET_BIND_SERVICE']", NOT "add: ["NET_BIND_SERVICE"]".

output example:
{{
    "Line number": "The line number where the misconfiguration occurs.",
    "Line text": "Text of the misconfiguration as it appears in the manifest.",
    "Reasoning": "Explanation of why this misconfiguration occurs within the manifest.",
    "Remediation": "Description of how to fix the misconfiguration.",
    "Error number": "{formatted_error_id}"
}}"""

    # 3. Few-Shot Examples (Q & A)
    # 论文文本中只有 Answer，这里我根据 Answer 手动补全了对应的 Input YAML (Q)，
    # 这样模型才能学会 "看到这个输入 -> 输出这个JSON" 的逻辑。
    
    few_shot_examples = """
Q:
LLM_error_134: Indicates when objects use deprecated API versions under extensions/v1beta.
KCF:
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: test-ingress

A:
{
    "Line number": 1,
    "Line text": "apiVersion: extensions/v1beta1",
    "Reasoning": "The API version 'extensions/v1beta1' for Ingress is deprecated and not supported in newer versions of K8s. Resources should use the current API version to ensure compatibility and access to the latest features.",
    "Remediation": "Update the apiVersion from 'extensions/v1beta1' to 'networking.k8s.io/v1' for the Ingress resource. This change ensures compatibility with newer K8s versions and leverages the latest features and fixes.",
    "Error number": "LLM_error_153"
}

Q:
LLM_error_18: The default namespace should not be used
KCF:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: gateway-ingress
spec:
  rules:
  - http:
      paths:
      - path: /test
        pathType: Prefix
        backend:
          service:
            name: test
            port:
              number: 80

A:
{
    "Line number": 4,
    "Line text": "name: gateway-ingress",
    "Reasoning": "The resource 'gateway-ingress' does not specify a namespace and therefore defaults to the 'default' namespace. Using the default namespace for production or shared environments can lead to conflicts and security issues, as it is accessible by all users in a cluster.",
    "Remediation": "Add a namespace to the metadata section of the Ingress resource to properly isolate resources and enhance security. For example, add 'namespace: custom-namespace' below the 'name' field.",
    "Error number": "LLM_error_9"
}
"""

    # 4. 当前任务 (Current Task)
    current_task = f"""
Q:
{formatted_error_id}: {error_description}
KCF:
{yaml_content}

A:
"""
    
    # 5. 组合最终 Prompt
    # Mistral 格式: [INST] System + Examples + Task [/INST]
    # 注意：我们把 System Prompt 和 Few-Shot 都放在 INST 里作为上下文
    full_prompt = f"[INST] {system_instruction}\n\n{few_shot_examples}\n\n{current_task} [/INST]"
    
    return full_prompt

In [37]:
# Cell 6: 解析与推理函数
import json
import re
def parse_llm_output(output_text):
    """
    从 LLM 的回复中提取 JSON。
    LLM 可能会在 JSON 前后说废话，需要清洗。
    """
    clean_text = output_text.strip()

    # 1. 尝试去除 Markdown (```json ... ```)
    match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", clean_text, re.DOTALL)
    if match:
        clean_text = match.group(1)
    # 2. 查找最外层的 {}
    # 这里的逻辑是：从左边找第一个 {，从右边找最后一个 }
    # 只要 Prompt 被成功剥离，这个逻辑就是无敌的
    start = clean_text.find('{')
    end = clean_text.rfind('}')
    
    if start != -1 and end != -1:
        json_str = clean_text[start : end + 1]
        def fix_nested_quotes(match):
            prefix = match.group(1) # "Remediation": "
            content = match.group(2) # 中间的乱七八糟的内容
            suffix = match.group(3) # ", "Error number"

            # 1. 把转义的双引号 \" 变成单引号 '
            content = content.replace('\\"', "'")
            # 2. 把剩下的没转义的双引号 " 也变成单引号 '
            content = content.replace('"', "'")
            
            return f"{prefix}{content}{suffix}"

        # 正则解释：
        # ("Remediation"\s*:\s*")  -> 捕获组1: 键名开头
        # (.*?)                    -> 捕获组2: 内容 (非贪婪匹配)
        # ("\s*,\s*"Error number") -> 捕获组3: 下一个键名 (作为锚点)
        pattern = r'("Remediation"\s*:\s*")(.*?)("\s*,\s*"Error number")'
        
        # 执行替换 (flags=re.DOTALL 让 . 能匹配换行符)
        json_str = re.sub(pattern, fix_nested_quotes, json_str, flags=re.DOTALL)
        json_str = json_str.replace('["', "['").replace('"]', "']")

        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            return {"error": f"JSONDecodeError: {str(e)}", "raw": output_text}
    else:
        return {"error": "No JSON braces found", "raw": output_text}

def genkube_resolve(yaml_content, detected_labels, model, tokenizer):
    results = []
    
    for label in detected_labels:
        # 解码标签: "Deployment+52" -> uid="52"
        parts = label.split('+')
        if len(parts) != 2:
            continue
        
        resource_kind, uid = parts
        
        # 获取描述
        error_desc = umi_map.get(uid, "Misconfiguration detected.")
        
        print(f"🔍 正在分析错误: {label} (ID: {uid})...")
        
        # 传入 uid (例如 "52")，函数内部会自动转为 "LLM_error_52"
        prompt = construct_prompt(yaml_content, uid, error_desc)
        # ----------------
        
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        input_len = inputs.input_ids.shape[1]

        with torch.no_grad():
            outputs = model.generate(
                **inputs, 
                max_new_tokens=512,
                do_sample=False, 
                pad_token_id=tokenizer.eos_token_id
            )
        
        generated_ids = outputs[0][input_len:]
        generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
        
        # 解析 JSON
        analysis = parse_llm_output(generated_text)
        if "error" in analysis:
            # 只有出错时才打印 raw，保持清爽
            print(f"⚠️ 解析失败！模型生成内容:\n{generated_text}\n" + "-"*20)
        
        results.append({
            "label": label,
            "description": error_desc,
            "analysis": analysis
        })
        
    return results

In [38]:
# Cell 7a: 加载 GenKubeDetect (检测模型)
import torch
from peft import PeftModel
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# --- 配置检测模型路径 ---
DETECT_BASE_PATH = "/ssd_2t_1/wyq_workspace/genkubesect_structural_model"
DETECT_LORA_PATH = "/ssd_2t_1/wyq_workspace/genkubesect_detection_model"

print("⏳ 正在加载检测模型 GenKubeDetect (CodeT5p) ...")

# 1. 加载 Tokenizer
detect_tokenizer = AutoTokenizer.from_pretrained(DETECT_BASE_PATH, trust_remote_code=True)

# 2. 加载基础模型
detect_base_model = AutoModelForSeq2SeqLM.from_pretrained(
    DETECT_BASE_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto" # 自动分配到 GPU
)

# 3. 加载 LoRA 适配器
detect_model = PeftModel.from_pretrained(detect_base_model, DETECT_LORA_PATH)
detect_model.eval()

print("✅ 检测模型加载完成！现在显存里有两个模型了 (Mistral + CodeT5p)。")

def run_detection(yaml_content):
    """
    使用 GenKubeDetect 模型检测 YAML
    返回: 原始标签字符串 (e.g., "Deployment+10, Service+52")
    """
    inputs = detect_tokenizer(
        yaml_content, 
        return_tensors="pt", 
        max_length=512, 
        truncation=True
    ).to(detect_model.device)
    
    with torch.no_grad():
        outputs = detect_model.generate(
            input_ids=inputs["input_ids"],
            max_new_tokens=128,
            num_beams=5,
            early_stopping=True
        )
    
    return detect_tokenizer.decode(outputs[0], skip_special_tokens=True)

⏳ 正在加载检测模型 GenKubeDetect (CodeT5p) ...


/home/wyq/.conda/envs/genkubesect/lib/python3.13/site-packages/accelerate/utils/modeling.py:1598: UserWarning: The following device_map keys do not match any submodules in the model: ['decoder.embed_tokens']
  warnings.warn(


✅ 检测模型加载完成！现在显存里有两个模型了 (Mistral + CodeT5p)。


In [39]:
# Cell 7b: 运行完整流水线 (Detect -> Resolve)

# 1. 定义待检测的 YAML (就是你之前提供的那个有问题的)
target_yaml = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:latest
        ports:
        - containerPort: 80
"""

print("="*50)
print("阶段 1: GenKubeDetect (检测中...)")
print("="*50)

# 1. 运行检测模型
raw_labels_str = run_detection(target_yaml)
print(f"📥 模型原始输出: {raw_labels_str}")

# 2. 清洗标签 (String -> List)
# 你的输出结果很长: "Deployment+10, Deployment+11, ..."
# 我们需要把它切割成列表，去除空格
if raw_labels_str.strip():
    detected_labels = [label.strip() for label in raw_labels_str.split(',')]
else:
    detected_labels = []

# 去重 (防止模型输出重复标签)
detected_labels = list(set(detected_labels))

print(f"解析到的标签 ({len(detected_labels)} 个): {detected_labels}")

# 3. 如果标签太多，为了演示方便，我们可以只取前 3 个
# 如果你想跑完所有 20 个错误，可以把下面这两行注释掉
# if len(detected_labels) > 3:
#     print(f"错误太多，为了节省时间，仅展示前 3 个错误的修复方案...")
#     detected_labels = detected_labels[:3]

print("\n" + "="*50)
print("阶段 2: GenKubeResolve (定位与修复中...)")
print("="*50)

# 4. 调用 Mistral 进行解释 (使用上一段代码定义的 genkube_resolve)
# 注意：这里假设你之前的 Notebook 单元格里已经定义了 `genkube_resolve`, `model` (Mistral) 和 `tokenizer` (Mistral)
resolution_reports = genkube_resolve(target_yaml, detected_labels, model, tokenizer)
# import json
# print(json.dumps(resolution_reports, indent=2, ensure_ascii=False))
# 5. 打印最终报告
# print("\n" + "="*50)
# print("最终安全审计报告")
# print("="*50)

# for res in resolution_reports:
#     print(f"\n[ID: {res['label']}]")
#     print(f"规则描述: {res['description']}")
    
#     analysis = res['analysis']
#     if "error" in analysis:
#         print(f"LLM 解析 JSON 失败: {analysis['error']}")
#     else:
#         print(f"行号: {analysis.get('Line number') or analysis.get('line')}")
#         print(f"原因: {analysis.get('Reasoning') or analysis.get('reason')}")
#         print(f"修复:\n{analysis.get('Remediation') or analysis.get('fix')}")
#     print("-" * 40)

with open(DETECTION_RESULT_PATH, "w", encoding="utf-8") as f:
    json.dump(resolution_reports, f, ensure_ascii=False, indent=2)

阶段 1: GenKubeDetect (检测中...)
📥 模型原始输出: Deployment+10, Deployment+11, Deployment+112, Deployment+12, Deployment+13, Deployment+14, Deployment+15, Deployment+16, Deployment+160, Deployment+17, Deployment+18, Deployment+19, Deployment+27, Deployment+29, Deployment+30, Deployment+32, Deployment+33, Deployment+34, Deployment+35, Deployment+4, Deployment+48, Deployment+8
解析到的标签 (22 个): ['Deployment+33', 'Deployment+35', 'Deployment+16', 'Deployment+14', 'Deployment+12', 'Deployment+112', 'Deployment+27', 'Deployment+29', 'Deployment+160', 'Deployment+10', 'Deployment+11', 'Deployment+15', 'Deployment+19', 'Deployment+4', 'Deployment+13', 'Deployment+8', 'Deployment+32', 'Deployment+48', 'Deployment+30', 'Deployment+18', 'Deployment+17', 'Deployment+34']

阶段 2: GenKubeResolve (定位与修复中...)
🔍 正在分析错误: Deployment+33 (ID: 33)...
🔍 正在分析错误: Deployment+35 (ID: 35)...
🔍 正在分析错误: Deployment+16 (ID: 16)...
🔍 正在分析错误: Deployment+14 (ID: 14)...
🔍 正在分析错误: Deployment+12 (ID: 12)...
🔍 正在分析错误: Deployment+112 (ID